In [ ]:
!pip install librosa
!pip install transformers
!pip install accelerate
!pip install peft

In [ ]:
import json
from inference_sqa_granite_onestep import infer_audio_context__text_question, infer_text_context__audio_question, \
    infer_audio_context__audio_question
import random
from ask_lisa import ask_lisa

In [ ]:
with open("../../system_prompts/eval_answer_prompt.txt", "r") as f:
    eval_prompt = f.read()

dataset_entries = []

with open("dataset/test.jsonl", encoding="UTF-8") as rf:
    for line in rf:
        entry = json.loads(line.strip())
        dataset_entries.append(entry)

In [ ]:
def evaluate(context: str, question: str, expected_answer: str, generated_answer: str):
    print("Context: " + context)
    print("Question: " + question)
    print("Expected Answer: " + expected_answer)
    print("Generated Answer: " + generated_answer)
    print("evaluation: " + ask_lisa(eval_prompt, f"""
        Kontext: {context}
        Frage: {question}
        Ground Truth Antwort: {expected_answer}
        Generierte Antwort: {generated_answer}
    """))

In [ ]:
def evaluate_random(amount: int = 5):
    for _ in range(amount):
        entry = random.choice(dataset_entries)
        generated_answer: str = infer_audio_context__text_question(entry)
        context: str = entry["text"]
        question: str = entry["question"]
        expected_answer: str = entry["answer"]
        evaluate(context, question, expected_answer, generated_answer)
        print("-" * 30)

In [ ]:
evaluate_random(20)

# Bewertung
- 0: Keine Antwort / irrelevant / reine Reproduktion
- 1: Korrekt
- 2: Falsch
- 3: Unvollständig / zu vage

In [ ]:
audio = "/training-1/asr/raw/m-ailabs/speakers/auf_zwei_planeten/wavs_new/zweiplaneten_27_lasswitz_f000023.wav"
question = "Wie wird der Kometenkapitän Jo in den erhaltenen Briefen beschrieben?"
context = "wir haben mit dem kometen kapitän jo briefe von ihr bekommen und sie schreibt dass er ein ganz netter beinahe zivilisierter mann sei"
expected_answer = "Er wird als ein ganz netter, beinahe zivilisierter Mann beschrieben."

In [ ]:
# 0
generated_answer = "beantworte die frage nachdem sie noch erschüttert standen begannen einige große tropfen zu fallen und nun kam der donner mit knarrenden schlägen"
evaluate(context, question, expected_answer, generated_answer)

In [ ]:
# 1 (genau gleich)
generated_answer = "Er wird als ein ganz netter, beinahe zivilisierter Mann beschrieben."
evaluate(context, question, expected_answer, generated_answer)

In [ ]:
# 1 (verkürzt)
generated_answer = "Als ein netter, beinahe zivilisierter Mann."
evaluate(context, question, expected_answer, generated_answer)

In [ ]:
# 1 (stark verkürzt)
generated_answer = "beinahe zivilisiert, nett"
evaluate(context, question, expected_answer, generated_answer)

In [ ]:
# 1 (umformuliert)
generated_answer = generated_answer = "Er ist ein freundlicher Mann, der ziemlich zivilisiert ist."
evaluate(context, question, expected_answer, generated_answer)

In [ ]:
# 2 (konträr)
generated_answer = "Er ist ein strenger und unfreundlicher Mann."
evaluate(context, question, expected_answer, generated_answer)

In [ ]:
# 2 (schwächer)
generated_answer = "Er ist ein gesunder, gut gutaussehender Mann."
evaluate(context, question, expected_answer, generated_answer)

In [ ]:
# 2 (halb wahr, halb erfunden)
generated_answer = generated_answer = "Er ist ein netter Mann, der gut aussieht"
evaluate(context, question, expected_answer, generated_answer)

In [ ]:
# 3 (unvollständig)
generated_answer = generated_answer = "Er scheint ein netter Mann zu sein."
evaluate(context, question, expected_answer, generated_answer)

In [ ]:
# 2 (hat nichts mit der Frage zu tun)
generated_answer = generated_answer = "Heute ist schönes Wetter."
evaluate(context, question, expected_answer, generated_answer)

In [ ]:
# 2 (hat nichts mit der Frage zu tun)
generated_answer = generated_answer = "Der Mann geht spazieren."
evaluate(context, question, expected_answer, generated_answer)